In [1]:
from brightway2 import *
from bw2io.importers.exiobase import Exiobase22Importer

In [2]:
if 'Class 4' not in projects:
    projects.current = "Class 1"
    projects.copy_project("Class 4")

projects.current = "Class 4"

create_core_migrations()

http://www.exiobase.eu/index.php/data-download/exiobase2-year-2007-full-data-set/78-mriot-ixi-fpa-coefficient-version2-2-2/file

In [4]:
fp = "/Users/cmutel/Downloads/mrIOT_IxI_fpa_coefficient_version2.2.2"

In [5]:
ex = Exiobase22Importer(fp)

Extracting metadata
Extracting emissions
Extracting resources
Extracting main IO table
Extracted 163 datasets and many exchanges in 0.78 seconds


In [6]:
ex.process_raw_data()
ex.biosphere

Aggregating `substances` and `extractions`


{'As - combustion': ('biosphere3', 'dc6dbdaa-9f13-43a8-8af5-6603688c6ad0'),
 'As - non combustion': ('biosphere3', 'dc6dbdaa-9f13-43a8-8af5-6603688c6ad0'),
 'Benzo(a)pyrene - combustion': ('biosphere3',
  '5e13c2ab-5466-4ff5-816d-702dfdf25f76'),
 'Benzo(a)pyrene - non combustion': ('biosphere3',
  '5e13c2ab-5466-4ff5-816d-702dfdf25f76'),
 'Benzo(b)fluoranthene - combustion': ('biosphere3',
  '35357464-0d86-4bbd-940f-7d0dd8e5df57'),
 'Benzo(b)fluoranthene - non combustion': ('biosphere3',
  '35357464-0d86-4bbd-940f-7d0dd8e5df57'),
 'Benzo(k)fluoranthene - combustion': ('biosphere3',
  'cc5f1637-8aa5-442f-a8c6-43c8739944a0'),
 'Benzo(k)fluoranthene - non combustion': ('biosphere3',
  'cc5f1637-8aa5-442f-a8c6-43c8739944a0'),
 'CH4 - combustion': ('biosphere3', '0795345f-c7ae-410c-ad25-1845784c75f5'),
 'CH4 - non combustion': ('biosphere3',
  'da1157e2-7593-4dfd-80dd-a3449b37a4d8'),
 'CO - combustion': ('biosphere3', 'ba2f3f82-c93a-47a5-822a-37ec97495275'),
 'CO - non combustion': ('biosph

In [6]:
ex.biosphere

[{'categories': ['air'],
  'description': None,
  'exiobase-code': '124-38-9c',
  'name': 'Carbon dioxide, fossil',
  'synonym': 'E_CO2_c'},
 {'categories': ['air'],
  'description': None,
  'exiobase-code': '74-82-8c',
  'name': 'Methane, fossil',
  'synonym': 'E_CH4_c'},
 {'categories': ['air'],
  'description': None,
  'exiobase-code': '10102-44-0c',
  'name': 'Dinitrogen monoxide',
  'synonym': 'E_N2O_c'},
 {'categories': ['air'],
  'description': None,
  'exiobase-code': '7446-09-5c',
  'name': 'Sulfur oxides',
  'synonym': 'E_SOx_c'},
 {'categories': ['air'],
  'description': None,
  'exiobase-code': '11104-93-1c',
  'name': 'Nitrogen oxides',
  'synonym': 'E_NOx_c'},
 {'categories': ['air'],
  'description': None,
  'exiobase-code': '7664-41-7c',
  'name': 'Ammonia',
  'synonym': 'E_NH3_c'},
 {'categories': ['air'],
  'description': None,
  'exiobase-code': '630-08-0c',
  'name': 'Carbon monoxide, fossil',
  'synonym': 'E_CO_c'},
 {'categories': ['air'],
  'description': None,
 

In [12]:
{x[2] for x in ex.outputs['emissions']}

{'CH4 - combustion', 'CO2 - combustion', 'N2O - combustion'}

In [14]:
ex.outputs['substances']

[('CO2 - combustion', '124-38-9c', 'E_CO2_c', ''),
 ('CH4 - combustion', '74-82-8c', 'E_CH4_c', ''),
 ('N2O - combustion', '10102-44-0c', 'E_N2O_c', ''),
 ('SOx - combustion', '7446-09-5c', 'E_SOx_c', ''),
 ('NOx - combustion', '11104-93-1c', 'E_NOx_c', ''),
 ('NH3 - combustion', '7664-41-7c', 'E_NH3_c', ''),
 ('CO - combustion', '630-08-0c', 'E_CO_c', ''),
 ('Benzo(a)pyrene - combustion', '50-32-8c', 'E_benzo_a_pyrene_c', ''),
 ('Benzo(b)fluoranthene - combustion',
  '205-99-2c',
  'E_benzo_b_fluoranthene_c',
  ''),
 ('Benzo(k)fluoranthene - combustion',
  '207-08-9c',
  'E_benzo_k_fluoranthene_c',
  ''),
 ('Indeno(1,2,3-cd)pyrene - combustion', '193-39-5c', 'E_indeno_pyrene_c', ''),
 ('PAH - combustion', 'PAHsc', 'E_PAHs_c', ''),
 ('PCBs - combustion', 'PCBsc', 'E_PCBs_c', ''),
 ('PCDD_F - combustion', 'PCDD_Fc', 'E_PCDD_F_c', ''),
 ('HCB - combustion', '118-74-1c', 'E_HCB_c', 'Hexachlorobenzene'),
 ('NMVOC - combustion',
  'NMVOCc',
  'E_NMVOC_c',
  'Non methane volatile organic com

In [16]:
len(ex.outputs['emissions'])

19555

In [17]:
len(ex.outputs['substances'])

85

In [18]:
19555 / 85.

230.05882352941177

In [22]:
ex.outputs['emissions'][-10:]

[('WM',
  'Landfill of waste: Food',
  'N2O - combustion',
  'air',
  'kg/M.EUR',
  8.691877866556435),
 ('WM',
  'Landfill of waste: Paper',
  'N2O - combustion',
  'air',
  'kg/M.EUR',
  6.878722674952837),
 ('WM',
  'Landfill of waste: Plastic',
  'N2O - combustion',
  'air',
  'kg/M.EUR',
  3.3932673601739123),
 ('WM',
  'Landfill of waste: Inert/metal/hazardous',
  'N2O - combustion',
  'air',
  'kg/M.EUR',
  7.2995257069173745),
 ('WM',
  'Landfill of waste: Textiles',
  'N2O - combustion',
  'air',
  'kg/M.EUR',
  2.0260027101322664),
 ('WM',
  'Landfill of waste: Wood',
  'N2O - combustion',
  'air',
  'kg/M.EUR',
  2.4062309925875556),
 ('WM',
  'Activities of membership organisation n.e.c.',
  'N2O - combustion',
  'air',
  'kg/M.EUR',
  325.0168383861509),
 ('WM',
  'Recreational, cultural and sporting activities',
  'N2O - combustion',
  'air',
  'kg/M.EUR',
  87.55573741911232),
 ('WM',
  'Other service activities',
  'N2O - combustion',
  'air',
  'kg/M.EUR',
  36.9013106

In [23]:
ex.biosphere

{('EXIOBASE 2.2 biosphere', 'CH4 - combustion'): {'categories': ['air'],
  'code': 'CH4 - combustion',
  'database': 'EXIOBASE 2.2 biosphere',
  'exiobase_code': '74-82-8c',
  'name': 'CH4 - combustion',
  'synonym': 'E_CH4_c',
  'type': 'emission',
  'unit': 'kg'},
 ('EXIOBASE 2.2 biosphere', 'CO2 - combustion'): {'categories': ['air'],
  'code': 'CO2 - combustion',
  'database': 'EXIOBASE 2.2 biosphere',
  'exiobase_code': '124-38-9c',
  'name': 'CO2 - combustion',
  'synonym': 'E_CO2_c',
  'type': 'emission',
  'unit': 'kg'},
 ('EXIOBASE 2.2 biosphere', 'N2O - combustion'): {'categories': ['air'],
  'code': 'N2O - combustion',
  'database': 'EXIOBASE 2.2 biosphere',
  'exiobase_code': '10102-44-0c',
  'name': 'N2O - combustion',
  'synonym': 'E_N2O_c',
  'type': 'emission',
  'unit': 'kg'}}

In [4]:
from bw2io.extractors.exiobase import ExiobaseDataExtractor

In [5]:
X = ExiobaseDataExtractor._extract_csv(fp, "mrEmissions_version2.2.2.txt")

In [26]:
len(X)

19555

In [6]:
len(X)

267948